In [1]:
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
import os
import pandas as pd
import numpy as np

# Read the CSV and Perform Basic Data Cleaning

In [2]:
input_data_file = "output_data_npi.txt"
#df = pd.read_csv(os.path.join("..", "Data", input_data_file),sep='\t')
df = pd.read_csv(os.path.join("..", "Data", input_data_file),sep='\t', nrows=2000000)

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
#df = df.dropna()
df.head()

C:\Users\gleix\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,npi,nppes_provider_gender,nppes_entity_code,nppes_provider_city,nppes_provider_zip,nppes_provider_state,nppes_provider_country,provider_type,medicare_participation_indicator,place_of_service,...,hcpcs_drug_indicator,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_Medicare_allowed_amt,average_submitted_chrg_amt,average_Medicare_payment_amt,year,leie_code,leie
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012,None,Not
1,1003000126,M,I,CUMBERLAND,2.15022e+08,MD,US,Internal Medicine,Y,F,...,N,115.0,112.0,115.0,135.25,199.0,108.115652,2012,None,Not
2,1003000126,M,I,CUMBERLAND,2.15022e+08,MD,US,Internal Medicine,Y,F,...,N,93.0,88.0,93.0,198.59,291.0,158.870000,2012,None,Not
3,1003000126,M,I,CUMBERLAND,2.15022e+08,MD,US,Internal Medicine,Y,F,...,N,111.0,83.0,111.0,38.75,58.0,30.720721,2012,None,Not
4,1003000126,M,I,CUMBERLAND,2.15022e+08,MD,US,Internal Medicine,Y,F,...,N,544.0,295.0,544.0,70.95,105.0,56.655662,2012,None,Not


In [3]:
df['nppes_provider_gender'] = df['nppes_provider_gender'].fillna('O')

In [4]:
df.columns

Index(['npi', 'nppes_provider_gender', 'nppes_entity_code',
       'nppes_provider_city', 'nppes_provider_zip', 'nppes_provider_state',
       'nppes_provider_country', 'provider_type',
       'medicare_participation_indicator', 'place_of_service', 'hcpcs_code',
       'hcpcs_description', 'hcpcs_drug_indicator', 'line_srvc_cnt',
       'bene_unique_cnt', 'bene_day_srvc_cnt', 'average_Medicare_allowed_amt',
       'average_submitted_chrg_amt', 'average_Medicare_payment_amt', 'year',
       'leie_code', 'leie'],
      dtype='object')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 22 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   npi                               500000 non-null  int64  
 1   nppes_provider_gender             500000 non-null  object 
 2   nppes_entity_code                 499999 non-null  object 
 3   nppes_provider_city               499999 non-null  object 
 4   nppes_provider_zip                499999 non-null  object 
 5   nppes_provider_state              499999 non-null  object 
 6   nppes_provider_country            499999 non-null  object 
 7   provider_type                     499999 non-null  object 
 8   medicare_participation_indicator  499999 non-null  object 
 9   place_of_service                  499999 non-null  object 
 10  hcpcs_code                        499999 non-null  object 
 11  hcpcs_description                 499999 non-null  o

# Select your features (columns)

In [6]:
# Set features. This will also be used as your x values.
selected_features = df[['line_srvc_cnt', 'place_of_service', 'hcpcs_drug_indicator', 'nppes_entity_code', 'nppes_provider_gender',
                       'medicare_participation_indicator', 'leie']]
selected_features = selected_features.dropna()
selected_features

,line_srvc_cnt,place_of_service,hcpcs_drug_indicator,nppes_entity_code,nppes_provider_gender,medicare_participation_indicator,leie
1,115.0,F,N,I,M,Y,Not
2,93.0,F,N,I,M,Y,Not
3,111.0,F,N,I,M,Y,Not
4,544.0,F,N,I,M,Y,Not
5,75.0,F,N,I,M,Y,Not
...,...,...,...,...,...,...,...
499995,29.0,F,N,I,M,Y,Not
499996,12.0,F,N,I,M,Y,Not
499997,14.0,O,N,I,M,Y,Not
499998,37.0,O,N,I,M,Y,Not


# Create a Train Test Split

Use `koi_disposition` for the y values

In [7]:
selected_features.info()

#output_data_file = "mike_look_test.txt"
#selected_features.to_csv(os.path.join("..", "Data", output_data_file), index= False, sep='\t')


<class 'pandas.core.frame.DataFrame'>
Int64Index: 499999 entries, 1 to 499999
Data columns (total 7 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   line_srvc_cnt                     499999 non-null  float64
 1   place_of_service                  499999 non-null  object 
 2   hcpcs_drug_indicator              499999 non-null  object 
 3   nppes_entity_code                 499999 non-null  object 
 4   nppes_provider_gender             499999 non-null  object 
 5   medicare_participation_indicator  499999 non-null  object 
 6   leie                              499999 non-null  object 
dtypes: float64(1), object(6)
memory usage: 30.5+ MB


In [8]:
disposition = selected_features["leie"]
disposition_names = ["Not", "On_List"]
selected_features = selected_features.drop("leie", axis=1)
feature_names = selected_features.columns
selected_features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 499999 entries, 1 to 499999
Data columns (total 6 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   line_srvc_cnt                     499999 non-null  float64
 1   place_of_service                  499999 non-null  object 
 2   hcpcs_drug_indicator              499999 non-null  object 
 3   nppes_entity_code                 499999 non-null  object 
 4   nppes_provider_gender             499999 non-null  object 
 5   medicare_participation_indicator  499999 non-null  object 
dtypes: float64(1), object(5)
memory usage: 26.7+ MB


In [9]:
data = pd.get_dummies(selected_features)
data

,line_srvc_cnt,place_of_service_F,place_of_service_O,hcpcs_drug_indicator_N,hcpcs_drug_indicator_Y,nppes_entity_code_I,nppes_entity_code_O,nppes_provider_gender_F,nppes_provider_gender_M,nppes_provider_gender_O,medicare_participation_indicator_N,medicare_participation_indicator_Y
1,115.0,1,0,1,0,1,0,0,1,0,0,1
2,93.0,1,0,1,0,1,0,0,1,0,0,1
3,111.0,1,0,1,0,1,0,0,1,0,0,1
4,544.0,1,0,1,0,1,0,0,1,0,0,1
5,75.0,1,0,1,0,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
499995,29.0,1,0,1,0,1,0,0,1,0,0,1
499996,12.0,1,0,1,0,1,0,0,1,0,0,1
499997,14.0,0,1,1,0,1,0,0,1,0,0,1
499998,37.0,0,1,1,0,1,0,0,1,0,0,1


In [10]:
data = data.drop(['nppes_entity_code_O', 'medicare_participation_indicator_Y', 'place_of_service_O', 'hcpcs_drug_indicator_Y'], axis=1)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 499999 entries, 1 to 499999
Data columns (total 8 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   line_srvc_cnt                       499999 non-null  float64
 1   place_of_service_F                  499999 non-null  uint8  
 2   hcpcs_drug_indicator_N              499999 non-null  uint8  
 3   nppes_entity_code_I                 499999 non-null  uint8  
 4   nppes_provider_gender_F             499999 non-null  uint8  
 5   nppes_provider_gender_M             499999 non-null  uint8  
 6   nppes_provider_gender_O             499999 non-null  uint8  
 7   medicare_participation_indicator_N  499999 non-null  uint8  
dtypes: float64(1), uint8(7)
memory usage: 11.0 MB


In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, disposition, test_size=0.50, random_state=42)

In [12]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 249999 entries, 315367 to 121959
Data columns (total 8 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   line_srvc_cnt                       249999 non-null  float64
 1   place_of_service_F                  249999 non-null  uint8  
 2   hcpcs_drug_indicator_N              249999 non-null  uint8  
 3   nppes_entity_code_I                 249999 non-null  uint8  
 4   nppes_provider_gender_F             249999 non-null  uint8  
 5   nppes_provider_gender_M             249999 non-null  uint8  
 6   nppes_provider_gender_O             249999 non-null  uint8  
 7   medicare_participation_indicator_N  249999 non-null  uint8  
dtypes: float64(1), uint8(7)
memory usage: 5.5 MB


In [13]:
y_train

315367    Not
196833    Not
187641    Not
418843    Not
72548     Not
         ... 
259179    Not
365839    Not
131933    Not
146868    Not
121959    Not
Name: leie, Length: 249999, dtype: object

In [14]:
# Support vector machine linear classifier
from sklearn.svm import SVC 
model = SVC(kernel='linear')

# Pre-processing
Scale the data using the MinMaxScaler and perform some feature selection

In [15]:
from sklearn.preprocessing import MinMaxScaler
X_minmax = MinMaxScaler().fit(X_train)

X_train_scale = X_minmax.transform(X_train)
X_test_scale = X_minmax.transform(X_test)


In [16]:
X_train_scale

array([[1.51818950e-04, 0.00000000e+00, 1.00000000e+00, ...,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [6.35435767e-04, 0.00000000e+00, 1.00000000e+00, ...,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.73437357e-05, 0.00000000e+00, 1.00000000e+00, ...,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [4.35459193e-04, 1.00000000e+00, 1.00000000e+00, ...,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.12220038e-05, 1.00000000e+00, 1.00000000e+00, ...,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [3.14248902e-05, 0.00000000e+00, 1.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

# Hyperparameter Tuning
Use GridSearchCV to tune the model's parameters

In [17]:
# Create the GridSearch estimator along with a parameter object containing the values to adjust
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1],
              'gamma': [1.0]}
grid = GridSearchCV(model, param_grid, verbose=3)

In [18]:
# Fit the model using the grid search estimator. 
# This will take the SVC model and try each combination of parameters
grid.fit(X_train_scale, y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] C=1, gamma=1.0 ..................................................
[CV] ...................... C=1, gamma=1.0, score=0.998, total=   2.1s
[CV] C=1, gamma=1.0 ..................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.0s remaining:    0.0s


[CV] ...................... C=1, gamma=1.0, score=0.998, total=   2.1s
[CV] C=1, gamma=1.0 ..................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    4.1s remaining:    0.0s


[CV] ...................... C=1, gamma=1.0, score=0.998, total=   2.1s
[CV] C=1, gamma=1.0 ..................................................
[CV] ...................... C=1, gamma=1.0, score=0.998, total=   2.1s
[CV] C=1, gamma=1.0 ..................................................
[CV] ...................... C=1, gamma=1.0, score=0.998, total=   2.1s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   10.4s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='linear', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [1], 'gamma': [1.0]}, pre_dispatch='2*n_jobs',
             refit=True, return_train_score=False, scoring=None, verbose=3)

In [19]:
print(f"Training Data Score: {grid.score(X_train_scale, y_train)}")
print(f"Testing Data Score: {grid.score(X_test_scale, y_test)}")

Training Data Score: 0.9983479933919736
Testing Data Score: 0.998324


In [20]:
# List the best parameters for this dataset
print(grid.best_params_)

{'C': 1, 'gamma': 1.0}


In [21]:
# List the best score
print(grid.best_score_)

0.9983479934398687


In [22]:
type(grid.cv_results_)

dict

In [23]:
print(grid.cv_results_.keys())

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_C', 'param_gamma', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])


In [24]:
import pandas as pd

cv_results = pd.DataFrame(grid.cv_results_)
cv_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,1.75205,0.01915,0.3359,0.005191,1,1,"{'C': 1, 'gamma': 1.0}",0.99836,0.99834,0.99834,0.99834,0.99836,0.998348,0.00001,1


In [25]:
cv_results_tiny = cv_results[['param_C', 'param_gamma', 'mean_test_score']]
cv_results_tiny.sort_values(by='mean_test_score', ascending=False).head(50)

,param_C,param_gamma,mean_test_score
0,1,1,0.998348


In [26]:
# Best index for the model to impact the model.
print(grid.best_index_)

0


In [27]:
# Make predictions with the hypertuned model
predictions = grid.predict(X_test_scale)

In [28]:
# Calculate classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions,
                            target_names=disposition_names))

C:\Users\gleix\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         Not       1.00      1.00      1.00    249581
     On_List       0.00      0.00      0.00       419

    accuracy                           1.00    250000
   macro avg       0.50      0.50      0.50    250000
weighted avg       1.00      1.00      1.00    250000



In [29]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'Medicare_SVC_Grid_10-21-20_Test.sav'
joblib.dump(grid, filename)

['Medicare_SVC_Grid_10-21-20_Test.sav']